<img src="images/rcnn.png" style="height: 400px">

In [10]:
import cv2
import cv2.ximgproc.segmentation as segmentation
import numpy as np
import random
from tensorflow.keras.preprocessing.image import img_to_array
from keras.models import load_model

In [3]:
image = cv2.imread("images/mnist.png")
cv2.imshow("Digits", image)

if cv2.waitKey(0) == ord("q"): 
    cv2.destroyAllWindows()

## 1-Create Selective Search Method

In [6]:
def selective_search(image):
    print("ss")

    # initiliaze selective search segmantation algorithm
    ss = segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(image)
    ss.switchToSelectiveSearchQuality()
    
    print("Segmentation started...")
    rects = ss.process()
    print("\nSegmentation has completed succesfully...")

    print("Length of segmentations:", len(rects))
    
    return rects[:100] # first 100 rectangles

## 2-Applying Selective Search

In [7]:
rects = selective_search(image)

ss
Segmentation started...

Segmentation has completed succesfully...
Length of segmentations: 877


## 3-Find Region of Interest (ROI)

In [14]:
proposals = []
boxes = []
output = image.copy()

for (x,y,w,h) in rects[:50]:
    
    color = [random.randint(0,255) for j in range(0,3)]
    cv2.rectangle(output, (x,y), (x+w,y+h),color, 2)
    
    roi = image[y:y+h,x:x+w]
    roi = cv2.resize(roi, dsize=(32,32), interpolation = cv2.INTER_LANCZOS4) # resmi küçültürken aradaki boşlukları nasıl dolduracağını belirtir.
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    
    roi = img_to_array(roi)
    
    proposals.append(roi)
    boxes.append((x,y,w+x,h+y))

proposals = np.array(proposals, dtype = "float64")    
boxes = np.array(boxes, dtype = "int32")   

## 4-Read Existing Model

### NOTE
- 4. Bölümdeki **2_real_time_digit_classification** klasöründe eğitilen model çağırılır.

In [15]:
model = load_model("model.h5")

## 5-Test Data

In [16]:
proba = model.predict(proposals)

In [17]:
number_list = []
idx = [] # indexes of probabilities
for i in range(len(proba)):
    
    max_prob = np.max(proba[i,:])
    if max_prob > 0.95:
        idx.append(i)
        number_list.append(np.argmax(proba[i])) # classification

## 6-Visualize Data

In [21]:
for i in range(len(number_list)):
    
    j = idx[i]

    # x,y,x+width, y+height
    cv2.rectangle(image, (boxes[j,0], boxes[j,1]), (boxes[j,2],boxes[j,3]),[0,0,255],2)
    cv2.putText(image, str(np.argmax(proba[j])),(boxes[j,0] + 5, boxes[j,1] + 5 ), cv2.FONT_HERSHEY_COMPLEX,1.5,(0,255,0))
    
cv2.imshow("Image",image)

if cv2.waitKey(0) == ord("q"): 
    cv2.destroyAllWindows()